# Extract metadata from files


This notebook offers visual and step by step review of the various BUV metadata files. 

When run it will go through each file and show the various options for each column, allowing to see how many different options there are. 


Running the export function will create an export folder in the current directory, with a file containing a row for each reviewed metadata file.


#### Import necessary libraries

In [1]:
import os
import pandas as pd

from ipyfilechooser import FileChooser
from IPython.display import display


In [2]:
# Constants

# NotPresent is the value generated when there is no value the spreadsheet, this is to distinguish from given NA values
NA_VAL = "NotPresent"

# SAMPLE_SHEET_LINES defines how many sample lines get printed out in the output each time a new file is processed
SAMPLE_SHEET_LINES = 3

#### Relevant Columns 
Check if these make sense.

These columns will be reviewed and transferred into the export excel sheet.

In the export excel sheet, FileName column is added to the start of the row

The columns marked as duplicates are columns that appear in the sheets, but not in the surveys_buv_doc file but they seem to refer to the same thing.




In [3]:
columns = ["SurveyID", # changed the order to SurveyID to know right away if it's multiple surveys or not
           "SurveyName",
           "EncoderName", # duplicate to Encoder
           "Encoder", 
           "DateEntry",
           "OfficeContact",
           "LinkToMarineReserve", # duplicate to Reserve
           "Reserve", 
           "Region",
           "SurveyLocationAcronym", # duplicate to ReserveCode
           "ReserveCode", 
           "SurveyStartDate",
           "ContractorName",
           "ContractNumber",
           "SurveyLeaderName",
           "StratifiedBy",
           "SiteSelectionDesign",
           "SurveyVerbatim",
           "FishMultiSpecies",
           "IsLongTermMonitoring",
           "RightsHolder",
           "RecordType",
           "IsMoreHabitatData",
           "LinkToContract",
           "LinkReport01",
           "LinkToOriginalData",
           "Vessel",
           "BaitSpecies",
           "BaitAmount"]

#### Current placeholder values for columns with no entries. 
Commented out ones will default to NA_VAL

In [4]:
empty_cell_solutions = {
# "SurveyName",
# "Encoder", # EncoderName?
# "DateEntry",
# "OfficeContact",
# "Reserve", # LinkToMarineReserve?
# "Region",
# "ReserveCode", # SurveyLocationAcronym? first 3 letters of SurveyID
# "SurveyID",
# "SurveyStartDate",
# "ContractorName",
# "ContractNumber",
# "SurveyLeaderName",
# "StratifiedBy": "check with Mon", 
"SiteSelectionDesign": "Non-random", 
# "SurveyVerbatim": NA_VAL,
"FishMultiSpecies": "TRUE", 
# "IsLongTermMonitoring",
# "RightsHolder", 
"RecordType": "FISH",
"IsMoreHabitatData": "TRUE", 
# "LinkToContract",
# "LinkReport01" : NA_VAL,
# "LinkToOriginalData": the number at the end of the filename - done in code 
"Vessel": NA_VAL,
"BaitSpecies" : "pilchard",
"BaitAmount": NA_VAL,
# Survey Name": This entry was removed
}
     

#### Select folder containing the files from which to extract metadata 

In [ ]:
folder_chooser = FileChooser(title='<b>Select a folder</b>')
display(folder_chooser)


In [ ]:
selected_folder = folder_chooser.selected_path
assert selected_folder != None, "Select folder in the cell above."
print(f"The selected folder is {selected_folder}")
all_files = os.listdir(selected_folder)
all_tab_files = [os.path.join(selected_folder, file) for file in all_files if file.endswith(".xlsx") or file.endswith(".xls") or file.endswith(".csv")]
all_tab_files

## Main function to run

In [33]:

def export_metadata_from_files_to_excel(all_tab_files, columns, export_excel_file_name=None, sheet_name="BUV"):
    print(f"Exporting following files:\n{all_tab_files}")
    
    problem_files = []
    list_of_metadata = []
    df_with_vals = pd.DataFrame()

    
    for file_name in all_tab_files:
        print(f"Working on file: '{file_name}'. Example rows:\n")

        # Extract data frame from Excel sheet
        current_file_df = get_df_from_sheet(file_name, sheet_name)
        if current_file_df.empty:
            print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!File not exported {file_name}!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            problem_files.append(file_name)
            continue

        # Display sample of data frame
        with pd.option_context('display.max_columns', None): 
            display(current_file_df.sample(min(SAMPLE_SHEET_LINES, current_file_df.shape[0])))


        # Create row with metadata
        file_metadata = [file_name]
        for column in columns:
            col_vals = current_file_df.get(column, pd.Series())
            column_val = extract_val(col_vals, column, file_name)
            file_metadata.append(column_val)
        list_of_metadata.append(file_metadata)

        # Export extracted metadata to Excel sheet in export folder
        # rewrite at each step to save progress after each file
        df_with_vals = pd.DataFrame(list_of_metadata, columns= ["FileName"] + columns)
        if not export_excel_file_name:
            export_excel_file_name = f"surveys_buv_doc_{sheet_name}.xlsx"
            
        # make export folder in folder containing all files
        path_to_export = os.path.join(selected_folder, "export")
        os.makedirs(path_to_export, exist_ok=True)
        export_location = os.path.join(path_to_export, export_excel_file_name)
        print(f"Exporting data to file: '{export_location}'")
        df_with_vals.to_excel(export_location)  
        
    print(f"Problem files: {problem_files}")
   
    return df_with_vals, problem_files
            

## Helper functions

In [27]:
def get_df_from_sheet(file_name, sheet_name):
    """Get a data frame from the given sheet and filename."""
    if file_name.endswith(".csv"):
        try: 
            return pd.read_csv(file_name)
        except: 
            print(f"Csv file {file_name} wasn't read.")
            return pd.DataFrame()
    try: 
        return pd.read_excel(file_name, sheet_name=sheet_name)
    except: 
        print(f"File {file_name} has no {sheet_name} sheet, it will try to process the first sheet.")
    try: 
        return pd.read_excel(file_name)
    except Exception as error:
        print("File wasn't processed because of:", error)
        return pd.DataFrame()
      

In [28]:
def extract_val(col_vals, column_name, file_name):
    """Extract value from column, populate for missing values."""
    # Populate for missing value
    if col_vals.empty:   
        print(f"\n{column_name} not present")
        unconfirmed_val = NA_VAL
        if column_name in empty_cell_solutions:
            unconfirmed_val = empty_cell_solutions[column_name]
        elif column_name == "LinkToOriginalData":
            unconfirmed_val = get_LinkToOriginalData(file_name)
        print(f"setting {column_name} as {unconfirmed_val}")

    # Extract values
    elif not col_vals.empty:
        unique_vals = list(col_vals.unique())
        
        # Add link to original data, as sometimes the value present in the cell doesn't make sense
        if column_name == 'LinkToOriginalData':
            link_val = get_LinkToOriginalData(file_name)
            if link_val not in unique_vals:
                unique_vals.append(link_val)
            print(f"If the 'LinkToOriginalData' entry doesn't make sense check the filename number after DOC-")
            
        if len(unique_vals) > 1:
            unconfirmed_val = deal_with_multiple_values(unique_vals, column_name)
        else: 
            unconfirmed_val = unique_vals[0]
      

    # Confirm value
    confirmed_val = confirm_val(unconfirmed_val, column_name)
    return confirmed_val

In [29]:
def get_LinkToOriginalData(file_name):
    """Get number at the end of filename that works as Link to Original Data"""
    doc_num_start = file_name.find("DOC-")
    unconfirmed_val = NA_VAL
    if doc_num_start != -1:
        unconfirmed_val = file_name[doc_num_start+4:]
        end_num = unconfirmed_val.find(".")
        if end_num != -1:
            unconfirmed_val = unconfirmed_val[:end_num]
    return unconfirmed_val
                

In [37]:
def deal_with_multiple_values(unique_vals, column_name):
    """Deal with the case when a column has multiple unique values."""
    print(f"Multiple values in column {column_name}")
    for i, value in enumerate(unique_vals): 
        print(f"{i+1}: {value}")
    print("""\nSelect the number in front of the correct value or
a: if you want to write your entry or
b: if you want to select all the values separated by a semicolon write.
\n""") 
    user_input = input("Enter your selection:\n")
    val = False
    while val is False:
        if user_input.lower() == "a":
            print(f"Write your entry for column {column_name}:\n")
            val = input("Write here:\n")
        elif user_input.lower() == "b":
            val = ";".join(map(str, unique_vals))
        elif not user_input.isdigit() or int(user_input) > len(unique_vals):
            print("Invalid entry, select again:\n")
            user_input = input("Enter your selection:\n")
        else:
            val = str(unique_vals[int(user_input)-1])
    return val
        

In [38]:

def confirm_val(unconfirmed_val, column_name):
    """Confirm the given value."""
    confirm = False
    while not confirm:
        print(f"\nThe value for {column_name} is:\n{unconfirmed_val}\nconfirm with 1 or add your value.\n")
        user_input = input("Enter your selection:\n")
        if user_input == "1":
            confirm = True
        else: 
            unconfirmed_val = user_input
    return str(unconfirmed_val)

# Run the function

This goes through each file in the selected folder, and offers for selection unique entries for each column. If anything looks suspicious, review the file manually.

Running the export function will create an export folder in the current directory, with a file containing a row for each reviewed metadata file. If you stop the function half way, the files that you have already processed have already been exported and have their corresponding row in the csv file in the export folder in your selected folder from above.

Some things to note: 
- sometimes the files still include examples in the first lines, so make sure to check if that is the case.


In [ ]:
metadata, problem_files = export_metadata_from_files_to_excel(all_tab_files=all_tab_files, columns=columns)

## Print the created metadata dataframe (that was exported to the excel file)

In [ ]:
metadata

In [43]:
problem_files

[]

## Notes

- Spaces were removed from the column names, and cases changed to match data files
- Added FileName as a column
- Created some columns to match the ones in the original files, it's easy to duplicate:  

    - "EncoderName", # duplicate to Encoder 
    - "LinkToMarineReserve", # duplicate to Reserve 
    - "SurveyLocationAcronym", # duplicate to ReserveCode 

- SurveyLocationAcronym is the start of SurveyID
- SurveyStartDate is the same as the one from SurveyID (didn't double check).
 
- Some files have a few example rows on top, to check which values are given by that 
- if there are two SurveyIDs in the same file, one can only review one at a time, so either do it manually, or run the file a second time, this time choosing the second option
- Some True and False are read as floats 1.0 and 0.0

